In [1]:
import torch
from torch import nn, optim
from torchvision import transforms, utils
from torch.utils.data import DataLoader, random_split
from models import resnet18
from dataset import PosterDataset, Resize, ToTensor
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
transformed_dataset = PosterDataset(csv_file='./data.txt',
                                    root_dir='../posters/',
                                    transform=transforms.Compose([
                                        Resize(),
                                        ToTensor()
                                    ]))
train_size = int(0.8*len(transformed_dataset)+1)
test_size = int(0.2*len(transformed_dataset))
train_dataset, test_dataset = random_split(transformed_dataset, [train_size, test_size])
data_loader1 = DataLoader(train_dataset, batch_size=8,shuffle=True)
data_loader2 = DataLoader(test_dataset, batch_size=8,shuffle=True)
print(len(data_loader1))
print(len(data_loader2))

278
70


In [3]:
device = torch.device('cuda')
model = resnet18().to(device)
criteon = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
train_loss = []
train_acc = []

for epoch in range(1):
    model.train()
    for idx, item in enumerate(data_loader1):
#         print(idx)
#         print(x)
#         print(label)
#         print(title)
        x, label = item['image'].to(device), item['label'].to(device)
        logits = model(x)
        loss = criteon(logits, label)
        train_loss.append(loss.item())

        # backporp
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print('epoch: ', epoch, ' [', idx, '/', len(data_loader1), '] ', 'loss: ', loss.item())

    print('epoch: ', epoch, 'loss: ', loss.item())

    model.eval()
    with torch.no_grad():
        # test
        total_correct = 0
        total_num = 0
        for idx, (x, label, title) in data_loader2:
            x, label = x.to(device), label.to(device)
            logits = model(x)
            pred = logits.argmax(dim=1)
            total_correct += torch.eq(pred, label).float().sum().item()
            total_num += x.size(0)

        acc = total_correct / total_num
        train_acc.append(acc)
        print('acc: ', acc, '\n')

    # 保存神经网络
    if epoch % 100 == 0:
        torch.save(model, 'net.pkl')                      # 保存整个神经网络的结构和模型参数
        print('saved in net.pkl')

plot_curve(train_loss, 'loss')
plot_curve(train_acc, 'acc')

# 保存神经网络
torch.save(model, 'net.pkl')                      # 保存整个神经网络的结构和模型参数
#     torch.save(net.state_dict(), 'net_params.pkl')  # 只保存神经网络的模型参数
print('saved in net.pkl')

RuntimeError: size mismatch, m1: [8 x 512], m2: [1024 x 19] at C:/w/1/s/tmp_conda_3.6_041836/conda/conda-bld/pytorch_1556684464974/work/aten/src\THC/generic/THCTensorMathBlas.cu:268